In [1]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Jason Matney (jmatney@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [2]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
from arcgis.gis import GIS
from time import time

In [3]:
# Login for ArcPro, AGOL or API Server 
#gis = GIS("https://www.arcgis.com", username="dpiazza_dewberry", password="*****")

In [4]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML"
hucs = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp"

In [5]:
arcpy.env.overwriteOutput = True

In [ ]:
st = time()

# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(hucs, "wbdhu12_a_IN_only.shp")

# All centroids of subwatersheds
all_centroids = os.path.join(hucs, "wbdhu12_a_IN_only_centroids.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Hydrography_HighRes_FlowLine_NHD_USGS.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "IN_EXTENT_MOSAIC\IN_EXTENT_MOSAIC.tif")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_ext_slope")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")


#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")


#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")



subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

In [ ]:
# GLOBAL VARIABLES
LOG_NAME = 'DataCollection_export.log'

In [ ]:
# #search cursor through each row of county subwatersheds file
# rows = arcpy.SearchCursor(all_subwatersheds)

# row_count = 0
# for row in rows:
    
#     row_count += 1
    
#     subwatershed_number = row.getValue("HUC12")
    
#     subwatershed_list.append(subwatershed_number)
    
#     print("------ Subwatershed:", subwatershed_number, "Number", row_count, "of 1590 ------")              
    

In [11]:
st = time()

row_count=366
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12']) as cursor:

    for row in cursor:
        if row[0] not in ['051202081102',	'051202080201',	'051201110302',	'051201060507',	'071200010303',	'050800030717',	'041000030604',	'051401010601',	'051401040403',	'051202050503',	'051202030203',	'040500011709',	'051201130801',	'040500010701',	'051202060310',	'051202040504',	'051201050405',	'051201040405',	'071200011009',	'051201010404',	'041000050104',	'051201111512',	'050902030505',	'051202030508',	'051202011107',	'051201100407',	'051201070103',	'050800030302',	'051201010502',	'051402010404',	'051201050504',	'051201061003',	'051201060901',	'071200010702',	'051402010801',	'040400010403',	'040500010103',	'051401041105',	'051201111902',	'051201111702',	'051202060303',	'051202011206',	'051201061208',	'051201030604',	'051201011403',	'071200010201',	'071200011403',	'041000030502',	'051202081304',	'050902030901',	'051202040401',	'051201080302',	'040500011802',	'051202070601',	'051402020105',	'051202020702',	'051201080802',	'051201020304',	'051201050104',	'071200010206',	'040500011009',	'051401010803',	'051201010401',	'051202030403',	'051201011602',	'040400010506',	'051201130903',	'051402011203',	'050902030204',	'051202021003',	'051401040803',	'051202081402',	'051202070202',	'051202011308',	'051202010701',	'051201060409',	'041000040602',	'071200010504',	'051402020204',	'041000040405',	'051202080502',	'051202080605',	'051201080509',	'051201011103',	'041000030601',	'051202060309',	'051202010703',	'051201080703',	'051201100105',	'051202010407',	'051201011502',	'051201061202',	'051401041404',	'051202081501',	'051202020904',	'051202070402',	'050800030408',	'051201100203',	'051202010605',	'051201040303',	'041000040604',	'040500012101',	'051202060503',	'051201100401',	'051202010204',	'051201100102',	'051201050206',	'051201061103',	'071200010406',	'051402010103',	'041000050203',	'051402010908',	'051202080403',	'051201040703',	'051202090503',	'051202070802',	'071200020505',	'051201020402',	'051401041004',	'051201020105',	'051202090702',	'051202020306',	'051202070302',	'051202020108',	'050800030602',	'051201081602',	'051201030507',	'051202040904',	'051202010801',	'071200010803',	'071200010203',	'050800030704',	'051202010601',	'051201060606',	'071200010403',	'051202070101',	'051202020102',	'051201080408',	'041000030806',	'051201130707',	'050800030709',	'051401010602',	'051202090303',	'051202030602',	'051202040805',	'051202010807',	'051201061306',	'040500011605',	'040500010802',	'051402010902',	'051202090103',	'051201110904',	'051202040201',	'051201010802',	'051201060803',	'051201060501',	'051201011003',	'040400010303',	'041000030203',	'051202090701',	'050902030502',	'051202011502',	'051201040506',	'051201060205',	'071200030305',	'071200030406',	'051401040804',	'051202020209',	'051202050104',	'051201100607',	'051201130701',	'051201010602',	'051201111105',	'051202030706',	'051201070403',	'071200020204',	'051201130601',
                         '051402011102',	'051402011102',	'051402011102',	'041000030506',	'051202080905',	'051202081005',	'071200020501',	'071200020305',	'040500011204',	'051202030501',	'051202010106',	'051202060201',	'051202020203',	'051202040702',	'051202050401',	'051202011005',	'050800030106',	'071200020706',	'051401040304',	'051201111506',	'051201111503',	'051202030806',	'071200020101',	'051201090802',	'051202081506',	'051202080702',	'051201050507',	'051201060904',	'040400010503',	'041000030703',	'051402010502',	'051402020102',	'051201020301',	'051201040603',	'051201060104',	'050800030204',	'041000040408',	'051202040302',	'051201100108',	'051201080202',	'071200011102',	'051401040601',	'051202011602',	'051202030504',	'051201011406',	'040500010806',	'050800011001',	'051202081202',	'051202080102',	'051201110602',	'051201050203',	'040500012203',	'071200010209',	'051401010703',	'051201110102',	'051202020604',	'051202030511',	'051202011110',	'051202010402',	'051201061309',	'071200020301',	'071200010309',	'051202090807',	'051202081307',	'051202020403',	'051201081403',	'051201080405',	'051202070503',	'050902031007',	'051401041102',	'051201111803',	'051201061303',	'051201130805',	'041000030305',	'051202080602',	'051201080602',	'051201070106',	'051201011202',	'050800020907',	'041000030401',	'051402020201',	'051202090403',	'051201080902',	'051202020907',	'051202081405',	'051202070104',	'051201040205',	'040500011805',	'041000050101',	'051202060306',	'051202011203',	'051202010503',	'051201061205',	'051201011506',	'071200011204',	'071200011003',	'051201130301',	'051401041201',	'040500012208',	'040500011102',	'051401010402',	'050800030714',	'050800030810',	'051201100404',	'051202010303',	'051201090701',	'051201060704',	'071200010705',	'071200010501',	'051201030103',	'051201100302',	'051202010608',	'051201040402',	'071200010808',	'040500012104',	'051402020604',	'051202070501',	'051201030201',	'051202030303',	'051201081303',	'051201080706',	'040400010204',	'051201080506',	'071200010104',	'040500011105',	'050800030807',	'050902030601',	'051202020105',	'051202040705',	'051201110701',	'051202011002',	'051201080501',	'051201040202',	'040500011903',	'051201130704',	'050800030605',	'050800030201',	'041000040503',	'051202090406',	'051202070705',	'051201030504',	'051201020405',	'071200021301',	'051201060701',	'051402010703',	'051202010103',	'040500011003',	'051202080304',	'051201050101',	'051401010806',	'040400010103',	'051201110703',	'051202040901',	'051202010804',	'071200011305',	'071200010806',	'071200011006',	'051202021006',	'051202060601',	'051202020309',	'051201081503',	'051201060406',	'051401010605',	'051202080807',	'051202011406',	'071200011310',	'071200010312',	'051201020102',	'051202081002',	'051201070205',	'051201050501',	'071200020403',	'051401040604',	'051401040301',	'051201090704',	'050800030305',	'051201130206',	'051202020206',	'051202050301',	'051401010302',	'050800030805',	'051201040509',	'051201060303',	'051201060101',	'071200030203',	'051402010905',	'051202081509',	'051202040204',	'051201010703']:
            subwatershed_number = row[0]
#             print(subwatershed_number)
            row_count += 1

#             subwatershed_number = row.getValue("HUC12")

            subwatershed_list.append(subwatershed_number)

            print("------ Subwatershed:", subwatershed_number, "Number", row_count, "of 1590 ------")              

            #select subwatershed from shapefile with all of them
            subwatershed_selection = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION",
                                                                             "HUC12 = " + "'"+subwatershed_number+"'")

            #select centroid from shapefile with all of them
            centroid_selection = arcpy.SelectLayerByAttribute_management(all_centroids, "NEW_SELECTION",
                                                                             "HUC12 = " + "'"+subwatershed_number+"'")
            #copy selected subwatershed to it's own file
            subwatershed = arcpy.CopyFeatures_management(subwatershed_selection, "subwatershed")

            #copy selected centroid to it's own file
            centroid = arcpy.CopyFeatures_management(centroid_selection, "centroid")

            #clip dem to subwatershed area
            # dem_clip = arcpy.Clip_management(dem_Indiana, "#", "dem_clip.tif", subwatershed, "#" , "ClippingGeometry", "NO_MAINTAIN_EXTENT")
            dem_clip =  os.path.join(path, "working_dsn\dem_clip.tif")
            arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection)
            print("clipped dem to subwatershed area")

            #calculate area of subwatershed
            arcpy.AddField_management(subwatershed, "AREA", "DOUBLE")

            area = arcpy.CalculateGeometryAttributes_management(subwatershed, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)

            #print area
            rows = arcpy.SearchCursor(area)

            for row in rows:
                area = row.getValue("AREA")

            area_list.append(area)
            print(area, " square kilometers")


            #calculate perimeter of subwatershed
            arcpy.AddField_management(subwatershed, "PERIMETER", "DOUBLE")
            perimeter = arcpy.CalculateGeometryAttributes_management(subwatershed, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                                     "KILOMETERS")

            print("perimeter calculated")

            #print perimeter
            rows = arcpy.SearchCursor(perimeter)

            for row in rows:
                perimeter = row.getValue("PERIMETER")

            perimeter_list.append(perimeter)
            print(perimeter, " kilometers")



            #get average slope using zonal statistics from slope dem
            avg_slope = arcpy.sa.ZonalStatistics(subwatershed, "FID", dem_slope, "MEAN")

            #get slope value
            avg_slope_result = arcpy.GetRasterProperties_management(avg_slope, "MAXIMUM")

            avg_slope_value = avg_slope_result.getOutput(0)

            avg_slope_list.append(avg_slope_value)

            print(avg_slope_value, " = avg slope (%)")


            #calculate circulatory ratio
            #ratio of area to the area of a circle having equal perimeter as the perimeter of drainage basin

            #area of a circle with same perimeter as above
            #C = 2(pi)r
            #r = C/(2pi)
            #A = (pi)r^2
            circle_radius = perimeter/(2*numpy.pi)
            print(circle_radius, "is the radius of a circle with the same perimeter.")
            circle_area = (numpy.pi*(circle_radius**2))
            print(circle_area, "is the area of a circle with the same perimeter.")

            #ratio of subwatershed perimeter to circle circumference
            circulatory_ratio = area / circle_area

            circulatory_ratio_list.append(circulatory_ratio)
            print(circulatory_ratio, " is the circulatory ratio")


            #calculate relief
            #elevation difference before basin outlet and highest point located in the perimeter of basin

            #find highest point on perimeter

            #use Raster Domain tool to get z-enabled polyline of perimeter
            perimeter_polyline = arcpy.RasterDomain_3d(dem_clip, "perimeter_polyline.shp", "LINE")
            print("created 3d polyline of subwatershed perimeter")

            #Convert each vertices of the polyline into points
            perimeter_points = arcpy.FeatureVerticesToPoints_management(perimeter_polyline, "perimeter_points.shp")
            print("created perimeter points")

            #get Z values into the attribute table for the points
            perimeter_points_Z = arcpy.AddZInformation_3d(perimeter_points, "Z")
            print("added z information")

            #get the max value from all of the points
            perimeter_stats = arcpy.Statistics_analysis(perimeter_points_Z, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

            # Get a list of field names to display
            field_names = [i.name for i in arcpy.ListFields(perimeter_stats) if i.type != 'OID']

            # Open a cursor to extract results from stats table
            cursor = arcpy.da.SearchCursor(perimeter_stats, field_names)

            # Create a pandas dataframe to display results
            df = pd.DataFrame(data=[row for row in cursor],
                                  columns=field_names)

            print(df)

            #get the values for max Z and min Z into a format to use them

            rows = arcpy.SearchCursor(perimeter_stats)

            for row in rows:
                max_z = row.getValue("MAX_Z")
                min_z = row.getValue("MIN_Z")


            print(max_z)
            print(min_z)

            relief = max_z - min_z
            print("The relief of the subwatershed is: ", relief," meters.")

            relief_list.append(relief)



            #find area covered by A and AE zones and X NFHL Zones

            #clip to subwatershed area
            nfhl_sfha_clip = arcpy.Clip_analysis(nfhl_sfha, subwatershed, "nfhl_sfha.shp")

            #select Zone A and Zone AE
            nfhl_sfha_aae_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                               "FLD_ZONE = 'AE' Or FLD_ZONE = 'A'", None)

            #copy features to new feature class
            nfhl_sfha_aae = arcpy.CopyFeatures_management(nfhl_sfha_aae_selection, "nfhl_sfha_aae")

            #add new field for area
            nfhl_sfha_aae_addfield = arcpy.AddField_management(nfhl_sfha_aae, "AREA", "DOUBLE")

            #calculate the area for each of the features
            aae_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_aae_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            aae_area_list = []  

            rows = arcpy.SearchCursor(aae_area)  
            for row in rows:  
                aae_area_feature = row.getValue("AREA")  
                aae_area_list.append(aae_area_feature)  

            aae_area_sum = sum(aae_area_list)

            aae_list.append(aae_area_sum)

            print(aae_area_sum, " = area of all AE and A zones in the subwatershed (square km)")



            #select Zone X, 0.2% chance flood area
            nfhl_sfha_x_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                               "FLD_ZONE = 'X' And ZONE_SUBTY = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'",
                                                                              None)

            #copy features to new feature class
            nfhl_sfha_x = arcpy.CopyFeatures_management(nfhl_sfha_x_selection, "nfhl_sfha_x")

            #add new field for area
            nfhl_sfha_x_addfield = arcpy.AddField_management(nfhl_sfha_x, "AREA", "DOUBLE")

            #calculate the area for each of the features
            x_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_x_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            x_area_list = []  

            rows = arcpy.SearchCursor(x_area)  
            for row in rows:  
                x_area_feature = row.getValue("AREA")  
                x_area_list.append(x_area_feature)  

            x_area_sum = sum(x_area_list)

            x_list.append(x_area_sum)

            print(x_area_sum, " = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)")


            #calculate area covered by lakes/reserviors 

            #clip to subwatershed area
            water_bodies_clip = arcpy.Clip_analysis(water_bodies, subwatershed, "water_bodies.shp")

            #copy features to new feature class
            water_bodies_copy = arcpy.CopyFeatures_management(water_bodies_clip, "water_bodies_copy.shp")

            #add new field for area
            water_bodies_addfield = arcpy.AddField_management(water_bodies_copy, "AREA", "DOUBLE")

            #calculate the area for each of the features
            water_bodies_area = arcpy.CalculateGeometryAttributes_management(water_bodies_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            water_bodies_area_list = []  

            rows = arcpy.SearchCursor(water_bodies_area)  
            for row in rows:  
                wb_area_values = row.getValue("AREA")  
                water_bodies_area_list.append(wb_area_values)  

            water_bodies_area_sum = sum(water_bodies_area_list)

            water_bodies_list.append(water_bodies_area_sum)
            print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


            #count all of the dams in the subwatershed

            #clip dams to subwatershed
            dams_clip = arcpy.Clip_analysis(dams, subwatershed, "dams.shp")

            #count number of dam points
            dams_count = arcpy.GetCount_management(dams_clip)

            dams_list.append(dams_count)

            print(dams_count, " = number of dams in the subwatershed")



            #count all of the bridges in the subwatershed

            #clip bridges to subwatershed
            bridges_clip = arcpy.Clip_analysis(bridges, subwatershed, "bridges.shp")

            #count number of dam points
            bridges_count = arcpy.GetCount_management(bridges_clip)

            bridges_list.append(bridges_count)

            print(bridges_count, " = number of bridges in the subwatershed")



            #calculate the kilometers of streets in the subwatershed

            #clip streets to subwatershed area 
            streets_clip = arcpy.Clip_analysis(streets, subwatershed, "streets.shp")

            #add field to calculate length of each street
            streets_addfield = arcpy.AddField_management(streets_clip, "LENGTH_KM", "DOUBLE")

            #calculate the area for each of the features
            streets_length = arcpy.CalculateGeometryAttributes_management(streets_addfield, "LENGTH_KM LENGTH_GEODESIC", 
                                                                          'KILOMETERS')
            #get the sum of all of the areas
            streets_lengths_list = []  

            rows = arcpy.SearchCursor(streets_length)  
            for row in rows:  
                streets_lengths_values = row.getValue("LENGTH_KM")  
                streets_lengths_list.append(streets_lengths_values)  

            streets_length_sum = sum(streets_lengths_list)

            streets_list.append(streets_length_sum)

            print(streets_length_sum, " = sum of all streets in the subwatershed (km)")


            #calculate km of railroads in subwatershed

            #clip railroads to subwatershed area 
            railroads_clip = arcpy.Clip_analysis(railroads, subwatershed, "railroads.shp")

            #add field to calculate length of each railroad
            railroads_addfield = arcpy.AddField_management(railroads_clip, "LENGTH_KM", "DOUBLE")

            #calculate the area for each of the features
            railroads_length = arcpy.CalculateGeometryAttributes_management(railroads_addfield, "LENGTH_KM LENGTH_GEODESIC", 'KILOMETERS', 
                                                                "",
                                                                None)
            #get the sum of all of the areas
            railroads_lengths_list = []  

            rows = arcpy.SearchCursor(railroads_length)  
            for row in rows:  
                railroads_lengths_values = row.getValue("LENGTH_KM")  
                railroads_lengths_list.append(railroads_lengths_values)  

            railroads_length_sum = sum(railroads_lengths_list)

            railroads_list.append(railroads_length_sum)

            print(railroads_length_sum, " = sum of all railroads in the subwatershed (km)")   


            #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level


            #clip to subwatershed
            population_clip = arcpy.Clip_analysis(population, subwatershed, "population.shp")

            #get total population
            #field = B01001_001E = Total Population (alias)
            #when clipped, field name changes to B01001_001

            #get the sum of all of the populations in each tract
            total_pop_list = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                total_pop_value = row.getValue('B01001_001') 
                total_pop_list.append(total_pop_value)  

            total_pop_sum = sum(total_pop_list)

            population_list.append(total_pop_sum)
            print(total_pop_sum, " = total population in the subwatershed") 

            #get the average of percentages of dependent age groups in each tract
            #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
            #when clipped field changes to B01001_61
            dependent_pop_list = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                dependent_pop_value = row.getValue('B01001__61') 
                dependent_pop_list.append(dependent_pop_value)  

            dependent_pop_avg_pct = mean(dependent_pop_list)

            dependent_population_list.append(dependent_pop_avg_pct)

            print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

            #find population density
            population_density = total_pop_sum / area

            population_density_list.append(population_density)

            print(population_density, " = population density of subwatershed (people/square km)")


            #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level

            #clip to subwatershed
            median_income_clip = arcpy.Clip_analysis(median_income, subwatershed, "median_income.shp")

            #get average median income 
            #field = B19049_001 = Median Household Income in past 12 months 
            #^(inflation-adjusted dollars to last year of 5-year range) (alias)

            #get the sum of all of the populations in each tract
            total_median_income_list = []  

            rows = arcpy.SearchCursor(median_income_clip)  
            for row in rows:  
                median_income_value = row.getValue('B19049_001') 
                total_median_income_list.append(median_income_value)  

            median_income_average = mean(total_median_income_list)

            avg_median_income_list.append(median_income_average)

            print(median_income_average, " = average median income in the subwatershed") 


            #get housing density

            #clip building footprints to subwatershed area
            building_footprints_clip = arcpy.Clip_analysis(building_footprints, subwatershed, "building_footprints.shp")

            #select buildings that are marked residential
            building_footprints_residential = arcpy.SelectLayerByAttribute_management(building_footprints_clip, "NEW_SELECTION",
                                                                                 "RES_NONRES = 'Res'")

            #get count of how many buildings there are 
            buildings_count = arcpy.GetCount_management(building_footprints_residential)

            buildings_count_number = buildings_count.getOutput(0)
            print(buildings_count_number, " = number of residential building footprints in the subwatershed")

            #divide number of buildings by subwatershed area
            housing_density = int(buildings_count_number) / area

            housing_density_list.append(housing_density)

            print(housing_density, " = housing density (buildings per square km)")


           #find total population from the 2013 5-year ACS estimates
            #get the sum of all of the 2013 populations in each tract
            #field = DP05_0001E = TotalPopulation from the 2013 ACS 5yr estimates
            total_pop_list_2013 = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                total_pop_value_2013 = row.getValue('DP05_0001E')
                if total_pop_value_2013 != '0':
                    total_pop_list_2013.append(int(total_pop_value_2013))  

            total_pop_sum_2013 = sum( total_pop_list_2013)

            print(total_pop_sum_2013, " = total 2013 population in the subwatershed") 

            #find population change between 2018 and 2013
            population_change = total_pop_sum - total_pop_sum_2013

            population_change_list.append(population_change)
            print(population_change, " = population change between 2013 and 2018")




            #calculate drainage density
            #the total length of all streams and tributaries divided by basin area

            #all streams in subwatershed
            streams_clip = arcpy.Clip_analysis(streams, subwatershed, "streams")
            print("streams clipped")

            #add field for length
            streams_addfield = arcpy.AddField_management(streams_clip, "LENGTH", "DOUBLE")

            #calculate geometry 
            streams_calculate = arcpy.CalculateGeometryAttributes_management(streams_addfield, [["LENGTH", "LENGTH_GEODESIC"]], 
                                                                             "KILOMETERS")
            print("length calculated")

            #get the sum of all of the areas
            stream_length_list = []  

            rows = arcpy.SearchCursor(streams_calculate)  
            for row in rows:  
                stream_length = row.getValue("LENGTH")  
                stream_length_list.append(stream_length)  

            stream_length_sum = sum(stream_length_list)

            print(stream_length_sum, " = length of all streams in subwatershed")

            print(area, " = subwatershed area")

            drainage_density = stream_length_sum / area

            drainage_density_list.append(drainage_density)

            print(drainage_density, "is the drainage density (streams/km)")

            #find watershed length
            #watershed length = distance from outlet to watershed boundary along the main channel
            #we are assuming that the longest stream above is the main channel

            watershed_length = max(stream_length_list)

            watershed_length_list.append(watershed_length)

            print(watershed_length, " = watershed length (longest stream in subwatershed)")


            #calculate shape factor 
            #watershed length squared divided by watershed area

            shape_factor = (watershed_length**2) / area

            shape_factor_list.append(shape_factor)

            print(shape_factor, " is the shape factor.")



            #calculate relief ratio
            #relief divided by watershed length
            #length is is kilometers, convert to meters

            watershed_length_meters = watershed_length * 1000

            relief_ratio = relief / watershed_length_meters

            relief_ratio_list.append(relief_ratio)

            print(relief_ratio, " = relief ratio")


            #calculate ruggedness number
            #product of relief and drainage density

            #relief is in meters, convert first to km
            relief_km = relief / 1000
            print(relief_km, " is the relief in km")

            ruggedness = relief_km * drainage_density

            ruggedness_list.append(ruggedness)
            print(ruggedness, " is the ruggedness number")


            #calculate elongation ratio
            #ratio of diameter of a circle having the same area as the basin to the max basin length


            #diameter of circle with same area
            #A = (pi)r^2
            #r = sqrt(A/pi)
            #d = r*2

            radius = numpy.sqrt(area/numpy.pi)
            print(radius, " miles is the radius of a circle with the same area.")

            diameter = radius*2
            print(diameter, " miles is the diameter of a circle with the same area.")

            elongation_ratio = diameter / watershed_length

            elongation_ratio_list.append(elongation_ratio)


            #get number of buildings inside the aae zone

            buildings_aae_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_aae )

            buildings_aae = arcpy.CopyFeatures_management(buildings_aae_select, "buildings_aae.shp")

            buildings_aae_count = arcpy.GetCount_management(buildings_aae)

            buildings_aae_list.append(buildings_aae_count)

            #get number of buildings inside the x zone

            buildings_x_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_x )

            buildings_x = arcpy.CopyFeatures_management(buildings_x_select, "buildings_x.shp")

            buildings_x_count = arcpy.GetCount_management(buildings_x)

            buildings_x_list.append(buildings_x_count)

            #get area of various land use codes
            #clip usa land use polygon to the subwatershed
            lu_subwatershed = arcpy.Clip_analysis(lu_usa, subwatershed, "lu_subwatershed.shp")

            lu_subwatershed_area = arcpy.AddGeometryAttributes_management(lu_subwatershed, "AREA_GEODESIC", '',
                                                              "SQUARE_KILOMETERS")


            #lu 21 = developed open space
            lu_21_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 21")

            lu_21 = arcpy.CopyFeatures_management(lu_21_select, "lu_21.shp")

            #get the sum of all of the areas
            lu_21_polygons_list = []  

            rows = arcpy.SearchCursor(lu_21)  
            for row in rows:  
                lu_21_polygon = row.getValue("AREA_GEO")  
                lu_21_polygons_list.append(lu_21_polygon)  

            lu_21_area_sum = sum(lu_21_polygons_list)

            lu_21_list.append(lu_21_area_sum)



            #lu 22 = developed low intensity
            lu_22_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 22")

            lu_22 = arcpy.CopyFeatures_management(lu_22_select, "lu_22.shp")

            #get the sum of all of the areas
            lu_22_polygons_list = []  

            rows = arcpy.SearchCursor(lu_22)  
            for row in rows:  
                lu_22_polygon = row.getValue("AREA_GEO")  
                lu_22_polygons_list.append(lu_22_polygon)  

            lu_22_area_sum = sum(lu_22_polygons_list)

            lu_22_list.append(lu_22_area_sum)


            #lu 23 = developed medium intensity
            lu_23_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 23")

            lu_23 = arcpy.CopyFeatures_management(lu_23_select, "lu_23.shp")

            #get the sum of all of the areas
            lu_23_polygons_list = []  

            rows = arcpy.SearchCursor(lu_23)  
            for row in rows:  
                lu_23_polygon = row.getValue("AREA_GEO")  
                lu_23_polygons_list.append(lu_23_polygon)  

            lu_23_area_sum = sum(lu_23_polygons_list)

            lu_23_list.append(lu_23_area_sum)



            #lu 24 = developed high intensity
            lu_24_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 24")

            lu_24 = arcpy.CopyFeatures_management(lu_24_select, "lu_24.shp")

            #get the sum of all of the areas
            lu_24_polygons_list = []  

            rows = arcpy.SearchCursor(lu_24)  
            for row in rows:  
                lu_24_polygon = row.getValue("AREA_GEO")  
                lu_24_polygons_list.append(lu_24_polygon)  

            lu_24_area_sum = sum(lu_24_polygons_list)

            lu_24_list.append(lu_24_area_sum)



            #lu 41 = deciduous forest
            lu_41_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 41")

            lu_41 = arcpy.CopyFeatures_management(lu_41_select, "lu_41.shp")

            #get the sum of all of the areas
            lu_41_polygons_list = []  

            rows = arcpy.SearchCursor(lu_41)  
            for row in rows:  
                lu_41_polygon = row.getValue("AREA_GEO")  
                lu_41_polygons_list.append(lu_41_polygon)  

            lu_41_area_sum = sum(lu_41_polygons_list)

            lu_41_list.append(lu_41_area_sum)


            #lu 82 = cultivated crops
            lu_82_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 82")

            lu_82 = arcpy.CopyFeatures_management(lu_82_select, "lu_82.shp")

            #get the sum of all of the areas
            lu_82_polygons_list = []  

            rows = arcpy.SearchCursor(lu_82)  
            for row in rows:  
                lu_82_polygon = row.getValue("AREA_GEO")  
                lu_82_polygons_list.append(lu_82_polygon)  

            lu_82_area_sum = sum(lu_82_polygons_list)

            lu_82_list.append(lu_82_area_sum)



            #get percent impervious indicator for subwatershed area
            avg_impervious_pct = arcpy.sa.ZonalStatistics(subwatershed, "FID", impervious_usa, "MEAN" )

            #get avg value  value
            avg_impervious_result = arcpy.GetRasterProperties_management(avg_impervious_pct, "MAXIMUM")

            avg_impervious_pct_value = avg_impervious_result.getOutput(0)
            print(avg_impervious_pct_value, " = avg impervious percent ")

            impervious_percent_list.append(avg_impervious_pct_value)



            #get distance from residential buildings to streams

            building_footprints_residential_copy = arcpy.CopyFeatures_management(building_footprints_residential, "buildings_res.shp")


            #project buildings and streams so they are in the same GCS
            output_coord_system = arcpy.SpatialReference(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML\working_dsn\NAD1983_ProjectionFile.prj")

            streams_project = arcpy.Project_management(streams_clip, "streams_project.shp",
                                                      output_coord_system)

            buildings_project = arcpy.Project_management(building_footprints_residential_copy, "buildings_res_project.shp",
                                                      output_coord_system)

            #use near tool to get distance to steams

            buildings_near = arcpy.Near_analysis(buildings_project, streams_project, "", "LOCATION", "", "GEODESIC")

            #get all the distances in to the streams
            dist_to_stream_list = []
            rows = arcpy.SearchCursor(buildings_near)  
            for row in rows:  
                dist_to_stream = row.getValue("NEAR_DIST")  
                dist_to_stream_list.append(dist_to_stream)  

            dist_to_stream_avg = mean(dist_to_stream_list)

            dist_to_stream_stdev = std(dist_to_stream_list)

            print("Distance to stream avg", dist_to_stream_avg)
            print("Distance to stream std", dist_to_stream_stdev)

            dist_to_stream_avg_list.append(dist_to_stream_avg)
            dist_to_stream_stdev_list.append(dist_to_stream_stdev)


         ## LOOP    

            #loop through all rainfall rasters in the same folder 


            arcpy.env.workspace = directory_rainfall

            rasters = arcpy.ListRasters("orb*")

            for raster in rasters:

                raster_name = os.path.basename(raster).rstrip(os.path.splitext(raster)[1])
                print("Raster name: ", raster_name)

                #get average rainfall for duration using zonal statistics
                #elevRaster = arcpy.sa.Raster(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML\working_dsn\dem_clip.tif")

                centroid_rainfall = arcpy.sa.ExtractValuesToPoints(centroid, raster, r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp\Centroid_Rainfall.shp", "NONE", "VALUE_ONLY")

                # Open a cursor on some fields in a table  
                rainfall_value = arcpy.da.SearchCursor(centroid_rainfall, ("RASTERVALU",)).next()[0]


        #         avg_rainfall = arcpy.sa.ZonalStatistics(subwatershed, "FID", raster, "MEAN" )

        #         #get avg value  value
        #         avg_result = arcpy.GetRasterProperties_management(avg_rainfall, "MAXIMUM")

                #rainfall_value = avg_result.getOutput(0)
                print(rainfall_value, " = avg rainfall in inches *1000")

                if raster_name == "orb100yr06h":
                    orb100yr06h_list.append(rainfall_value)

                elif raster_name == "orb100yr12h":
                    orb100yr12h_list.append(rainfall_value)

                elif raster_name == "orb100yr24h":
                    orb100yr24h_list.append(rainfall_value)

                elif raster_name == "orb25yr06h":
                    orb25yr06h_list.append(rainfall_value)

                elif raster_name == "orb25yr12h":
                    orb25yr12h_list.append(rainfall_value)

                elif raster_name == "orb25yr24h":
                    orb25yr24h_list.append(rainfall_value)

                elif raster_name == "orb2yr06h":
                    orb2yr06h_list.append(rainfall_value)

                elif raster_name == "orb2yr12h":
                    orb2yr12h_list.append(rainfall_value)

                elif raster_name == "orb2yr24h":
                    orb2yr24h_list.append(rainfall_value)

                elif raster_name == "orb50yr06h":
                    orb50yr06h_list.append(rainfall_value)

                elif raster_name == "orb50yr12h":
                    orb50yr12h_list.append(rainfall_value)

                elif raster_name == "orb50yr24h":
                    orb50yr24h_list.append(rainfall_value)

                elif raster_name == "orb100yr06ha_am":
                    orb100yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb100yr12ha_am":
                    orb100yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb100yr24ha_am":
                    orb100yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr06ha_am":
                    orb25yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr12ha_am":
                    orb25yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr24ha_am":
                    orb25yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr06ha_am":
                    orb2yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr12ha_am":
                    orb2yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr24ha_am":
                    orb2yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr06ha_am":
                    orb50yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr12ha_am":
                    orb50yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr24ha_am":
                    orb50yr24ha_am_list.append(rainfall_value)

                else:
                    continue 



            #set workspace environment back to the newly created folder
            arcpy.env.workspace = output_workspace
            print(round(((time()-st)/60), 2) , 'minutes to process.')  


    
    
#########################################################################################


    
    
print(round(((time()-st)/60), 2) , 'minutes to process.')    
    

------ Subwatershed: 051402011102 Number 178 of 1590 ------
clipped dem to subwatershed area
59.1980110246  square kilometers
perimeter calculated
42.8279426266  kilometers
3.1010601520538  = avg slope (%)
6.816278771479482 is the radius of a circle with the same perimeter.
145.9635980759174 is the area of a circle with the same perimeter.
0.40556694823191747  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       4541  158.0  110.0
158.0
110.0
The relief of the subwatershed is:  48.0  meters.
8.379877266461  = area of all AE and A zones in the subwatershed (square km)
0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
2.981490523923921  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
7  = number of bridges in the subwatershed
115.25277810353805  = sum of all streets in the subwatershed (km)
12.0069861444

0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
0.11153513608406458  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
3  = number of bridges in the subwatershed
34.11517672935875  = sum of all streets in the subwatershed (km)
6.19930622874  = sum of all railroads in the subwatershed (km)
7431  = total population in the subwatershed
37.8  = total percent of dependent population in the subwatershed
93.43658946280509  = population density of subwatershed (people/square km)
53027.0  = average median income in the subwatershed
53  = number of residential building footprints in the subwatershed
0.6664162618124976  = housing density (buildings per square km)
7508  = total 2013 population in the subwatershed
-77  = population change between 2013 and 2018
streams clipped
length calculated
104.31591743931358  = length of all streams in subwatershed
79.529871999  = subwatershed area
1.3116570518386506 is the drainage 

streams clipped
length calculated
16.773633401326542  = length of all streams in subwatershed
45.4726622953  = subwatershed area
0.3688729129690796 is the drainage density (streams/km)
2.64683795213  = watershed length (longest stream in subwatershed)
0.15406511937524797  is the shape factor.
0.01003410199465644  = relief ratio
0.026558641975000002  is the relief in km
0.00979676362982112  is the ruggedness number
3.8045233551249087  miles is the radius of a circle with the same area.
7.609046710249817  miles is the diameter of a circle with the same area.
1.1055517196655  = avg impervious percent 
Distance to stream avg 1042.1560181787709
Distance to stream std 854.5117999914098
Raster name:  orb100yr06h
5051  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
5030  = avg rainfall in inches *1000
Raster name:  orb100yr12h
5742  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
5719  = avg rainfall in inches *1000
Raster name:  orb100yr24h
6726  = avg rainfall in i

Raster name:  orb100yr06h
4572  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4553  = avg rainfall in inches *1000
Raster name:  orb100yr12h
5232  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
5211  = avg rainfall in inches *1000
Raster name:  orb100yr24h
5721  = avg rainfall in inches *1000
Raster name:  orb100yr24ha_am
5698  = avg rainfall in inches *1000
Raster name:  orb25yr06h
3531  = avg rainfall in inches *1000
Raster name:  orb25yr06ha_am
3516  = avg rainfall in inches *1000
Raster name:  orb25yr12h
4042  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
4025  = avg rainfall in inches *1000
Raster name:  orb25yr24h
4505  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4486  = avg rainfall in inches *1000
Raster name:  orb2yr06h
1982  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1825  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2283  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2102  = avg

4465  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2010  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1850  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2360  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2173  = avg rainfall in inches *1000
Raster name:  orb2yr24h
2775  = avg rainfall in inches *1000
Raster name:  orb2yr24ha_am
2555  = avg rainfall in inches *1000
Raster name:  orb50yr06h
3710  = avg rainfall in inches *1000
Raster name:  orb50yr06ha_am
3695  = avg rainfall in inches *1000
Raster name:  orb50yr12h
4271  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
4253  = avg rainfall in inches *1000
Raster name:  orb50yr24h
4972  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
4952  = avg rainfall in inches *1000
8.2 minutes to process.
------ Subwatershed: 051202060201 Number 187 of 1590 ------
clipped dem to subwatershed area
44.0571057651  square kilometers
perimeter calculated
43.6082267247  kilometers
1.72

...

4576  = avg rainfall in inches *1000
Raster name:  orb25yr24h
5565  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
5542  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2133  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1964  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2483  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2286  = avg rainfall in inches *1000
Raster name:  orb2yr24h
2966  = avg rainfall in inches *1000
Raster name:  orb2yr24ha_am
2731  = avg rainfall in inches *1000
Raster name:  orb50yr06h
4685  = avg rainfall in inches *1000
Raster name:  orb50yr06ha_am
4666  = avg rainfall in inches *1000
Raster name:  orb50yr12h
5328  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
5306  = avg rainfall in inches *1000
Raster name:  orb50yr24h
6437  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
6411  = avg rainfall in inches *1000
700.42 minutes to process.
------ Subwatershed: 051402010905 Number 361 o

5186  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
5165  = avg rainfall in inches *1000
Raster name:  orb50yr24h
6232  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
6207  = avg rainfall in inches *1000
719.62 minutes to process.
------ Subwatershed: 051202040204 Number 363 of 1590 ------
clipped dem to subwatershed area
42.9209260005  square kilometers
perimeter calculated
34.3103290518  kilometers
0.018896382302046  = avg slope (%)
5.460658467703432 is the radius of a circle with the same perimeter.
93.67849443320137 is the area of a circle with the same perimeter.
0.4581726709015943  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       3831  255.0  248.0
255.0
248.0
The relief of the subwatershed is:  7.0  meters.
3.0780861233305  = area of all AE and A zones in the subwatershed (square km)
0  = area of all X zones, 0.2PCT zone subtype in the subwat

created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       4977  255.0  225.0
255.0
225.0
The relief of the subwatershed is:  30.0  meters.
7.754583989091308  = area of all AE and A zones in the subwatershed (square km)
0.27900569240757794  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
1.0036826985193914  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
12  = number of bridges in the subwatershed
120.82023134421698  = sum of all streets in the subwatershed (km)
14.406238380859  = sum of all railroads in the subwatershed (km)
22134  = total population in the subwatershed
39.5  = total percent of dependent population in the subwatershed
353.1607931744449  = population density of subwatershed (people/square km)
88158.8  = average median income in the subwatershed
1088  = number of residential building footprints in the subwatershed
17.359670325011116  = housing density (buildings 

2975  = total population in the subwatershed
34.2  = total percent of dependent population in the subwatershed
68.50339898504309  = population density of subwatershed (people/square km)
70231.0  = average median income in the subwatershed
38  = number of residential building footprints in the subwatershed
0.8750013988005504  = housing density (buildings per square km)
2948  = total 2013 population in the subwatershed
27  = population change between 2013 and 2018
streams clipped
length calculated
62.047795270122485  = length of all streams in subwatershed
43.4285020025  = subwatershed area
1.4287344119433512 is the drainage density (streams/km)
4.64480339503  = watershed length (longest stream in subwatershed)
0.4967751035308628  is the shape factor.
0.008181185890593452  = relief ratio
0.038  is the relief in km
0.054291907653847345  is the ruggedness number
3.7180265638572703  miles is the radius of a circle with the same area.
7.4360531277145405  miles is the diameter of a circle wit

KeyboardInterrupt: 

In [13]:
st = time()
outputs = {'subwatershed': subwatershed_list[0:189],
          'area': area_list[0:189],
          'perimeter': perimeter_list[0:189],
          'circulatory_ratio': circulatory_ratio_list[0:189],
          'relief': relief_list[0:189],
          'avg_slope': avg_slope_list[0:189],
           'watershed_length': watershed_length_list[0:189],
           'elongation_ratio': elongation_ratio_list[0:189],
           'drainage_density': drainage_density_list[0:189],
           'shape_factor': shape_factor_list[0:189],
           'relief_ratio': relief_ratio_list[0:189],
           'ruggedness': ruggedness_list[0:189],
           'aae_area': aae_list[0:189],
           'buildings_aae_count': buildings_aae_list[0:189],
           'x_area': x_list[0:189],
           'buildings_x_count': buildings_x_list[0:189],
           'water_bodies_area': water_bodies_list[0:189],
           'dams_count': dams_list[0:189],
           'bridges_count': bridges_list[0:189],
           'streets_km': streets_list[0:189],
           'railroads_km': railroads_list[0:189],
           'population': population_list[0:189],
           'population_density': population_density_list[0:189],
           'avg_median_income': avg_median_income_list[0:189],
           'housing_density': housing_density_list[0:189],
           'population_change': population_change_list[0:189],
           'dependent_population_pct': dependent_population_list[0:189],
           'dist_to_stream_avg (m)': dist_to_stream_avg_list[0:189],
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list[0:189],
           'lu_21_area' : lu_21_list[0:189],
           'lu_22_area' : lu_22_list[0:189],
           'lu_23_area' : lu_23_list[0:189],
           'lu_24_area': lu_24_list[0:189],
           'lu_41_area': lu_41_list[0:189],
           'lu_82_area': lu_82_list[0:189],
           'avg_impervious_percent': impervious_percent_list[0:189],
           'orb100yr06h': orb100yr06h_list[0:189],
           'orb100yr12h': orb100yr12h_list[0:189],
           'orb100yr24h': orb100yr24h_list[0:189],
           'orb25yr06h': orb25yr06h_list[0:189],
           'orb25yr12h': orb25yr12h_list[0:189],
           'orb25yr24h':orb25yr24h_list[0:189],
           'orb2yr06h': orb2yr06h_list[0:189],
           'orb2yr12h': orb2yr12h_list[0:189],
           'orb2yr24h': orb2yr24h_list[0:189],
           'orb50yr06h': orb50yr06h_list[0:189],
           'orb50yr12h': orb50yr12h_list[0:189],
           'orb50yr24h':orb50yr24h_list[0:189],
           'orb100yr06ha_am': orb100yr06ha_am_list[0:189],
           'orb100yr12ha_am': orb100yr12ha_am_list[0:189],
           'orb100yr24ha_am': orb100yr24ha_am_list[0:189],
           'orb25yr06ha_am': orb25yr06ha_am_list[0:189],
           'orb25yr12ha_am': orb25yr12ha_am_list[0:189],
           'orb25yr24ha_am': orb25yr24ha_am_list[0:189],
           'orb2yr06ha_am': orb2yr06ha_am_list[0:189],
           'orb2yr12ha_am': orb2yr12ha_am_list[0:189],
           'orb2yr24ha_am': orb2yr24ha_am_list[0:189],
           'orb50yr06ha_am': orb50yr06ha_am_list[0:189],
           'orb50yr12ha_am': orb50yr12ha_am_list[0:189],
           'orb50yr24ha_am': orb50yr24ha_am_list[0:189]
          }

# 
# 
# 
#  
#         
# 
            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                             'circulatory_ratio',
                                             'relief',
                                             'avg_slope',
                                              'watershed_length',
                                              'elongation_ratio',
                                              'drainage_density',
                                              'shape_factor',
                                              'relief_ratio',
                                              'ruggedness',
                                              'aae_area',
                                              'buildings_aae_count',
                                              'x_area',
                                              'buildings_x_count',
                                              'water_bodies_area',
                                              'dams_count',
                                              'bridges_count',
                                              'streets_km',
                                              'railroads_km',
                                              'population',
                                              'population_density',
                                              'avg_median_income',
                                              'housing_density',
                                              'population_change',
                                              'dependent_population_pct',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)',
                                              'lu_189_area',
                                              'lu_22_area',
                                              'lu_23_area',
                                              'lu_24_area',
                                              'lu_41_area',
                                              'lu_82_area',
                                              'avg_impervious_percent',
                                             'orb100yr06h',
                                              'orb100yr12h',
                                              'orb100yr24h',
                                              'orb25yr06h',
                                              'orb25yr12h',
                                              'orb25yr24h',
                                              'orb2yr06h',
                                              'orb2yr12h',
                                              'orb2yr24h',
                                              'orb50yr06h',
                                              'orb50yr12h',
                                              'orb50yr24h',
                                              'orb100yr06ha_am',
                                              'orb100yr12ha_am',
                                              'orb100yr24ha_am',
                                              'orb25yr06ha_am',
                                              'orb25yr12ha_am',
                                              'orb25yr24ha_am',
                                              'orb2yr06ha_am',
                                              'orb2yr12ha_am',
                                              'orb2yr24ha_am',
                                              'orb50yr06ha_am',
                                              'orb50yr12ha_am',
                                              'orb50yr24ha_am'
                                             ])    
    

#     
#  
# 
# 
# 
#




print(outputs_df)

outputs_df.to_excel(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\output\IN_subwatershed_data_178_366.xlsx")
print(round(((time()-st)/60), 2) , 'minutes to process.')

     subwatershed       area  perimeter  circulatory_ratio  relief  \
0    051402011102  59.198011  42.827943           0.405567    48.0   
1    051402011102  59.198011  42.827943           0.405567    48.0   
2    051402011102  59.198011  42.827943           0.405567    48.0   
3    041000030506  70.572820  64.592767           0.212559    15.0   
4    051202080905  79.529872  49.062317           0.415187   117.0   
..            ...        ...        ...                ...     ...   
184  071200030203  57.009648  40.921815           0.427808    44.0   
185  051402010905  58.285685  37.224962           0.528571    72.0   
186  051202081509  54.130025  38.027782           0.470377    52.0   
187  051202040204  42.920926  34.310329           0.458173     7.0   
188  051201010703  82.797399  50.349357           0.410430    13.0   

             avg_slope  watershed_length  elongation_ratio  drainage_density  \
0      3.1010601520538          3.857231          2.250780          1.139659   

In [12]:
len(orb50yr24ha_am_list) 

189

In [36]:
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12']) as cursor:

    for row in cursor:
        if row[0] not in ['051202080201',	'051201110302',	'051201060507',	'071200010303',	'050800030717',	'041000030604',	'051401010601',	'051401040403',	'051202050503',	'051202030203',	'040500011709',	'051201130801',	'040500010701',	'051202060310',	'051202040504',	'051201050405',	'051201040405',	'071200011009',	'051201010404',	'041000050104',	'051201111512',	'050902030505',	'051202030508',	'051202011107',	'051201100407',	'051201070103',	'050800030302',	'051201010502',	'051402010404',	'051201050504',	'051201061003',	'051201060901',	'071200010702',	'051402010801',	'040400010403',	'040500010103',	'051401041105',	'051201111902',	'051201111702',	'051202060303',	'051202011206',	'051201061208',	'051201030604',	'051201011403',	'071200010201',	'071200011403',	'041000030502',	'051202081304',	'050902030901',	'051202040401',	'051201080302',	'040500011802',	'051202070601',	'051402020105',	'051202020702',	'051201080802',	'051201020304',	'051201050104',	'071200010206',	'040500011009',	'051401010803',	'051201010401',	'051202030403',	'051201011602',	'040400010506',	'051201130903',	'051402011203',	'050902030204',	'051202021003',	'051401040803',	'051202081402',	'051202070202',	'051202011308',	'051202010701',	'051201060409',	'041000040602',	'071200010504',	'051402020204',	'041000040405',	'051202080502',	'051202080605',	'051201080509',	'051201011103',	'041000030601',	'051202060309',	'051202010703',	'051201080703',	'051201100105',	'051202010407',	'051201011502',	'051201061202',	'051401041404',	'051202081501',	'051202020904',	'051202070402',	'050800030408',	'051201100203',	'051202010605',	'051201040303',	'041000040604',	'040500012101',	'051202060503',	'051201100401',	'051202010204',	'051201100102',	'051201050206',	'051201061103',	'071200010406',	'051402010103',	'041000050203',	'051402010908',	'051202080403',	'051201040703',	'051202090503',	'051202070802',	'071200020505',	'051201020402',	'051401041004',	'051201020105',	'051202090702',	'051202020306',	'051202070302',	'051202020108',	'050800030602',	'051201081602',	'051201030507',	'051202040904',	'051202010801',	'071200010803',	'071200010203',	'050800030704',	'051202010601',	'051201060606',	'071200010403',	'051202070101',	'051202020102',	'051201080408',	'041000030806',	'051201130707',	'050800030709',	'051401010602',	'051202090303',	'051202030602',	'051202040805',	'051202010807',	'051201061306',	'040500011605',	'040500010802',	'051402010902',	'051202090103',	'051201110904',	'051202040201',	'051201010802',	'051201060803',	'051201060501',	'051201011003',	'040400010303',	'041000030203',	'051202090701',	'050902030502',	'051202011502',	'051201040506',	'051201060205',	'071200030305',	'071200030406',	'051401040804',	'051202020209',	'051202050104',	'051201100607',	'051201130701',	'051201010602',	'051201111105',	'051202030706',	'051201070403',	'071200020204',	'051201130601']:
            subwatershed_number = row[0]
            print(subwatershed_number)
            print(row_count)

051202081102
178
051402011102
178
041000030506
178
051202080905
178
051202081005
178
071200020501
178
071200020305
178
040500011204
178
051202030501
178
051202010106
178
051202060201
178
051202020203
178
051202040702
178
051202050401
178
051202011005
178
050800030106
178
071200020706
178
051401040304
178
051201111506
178
051201111503
178
051202030806
178
071200020101
178
051201090802
178
051202081506
178
051202080702
178
051201050507
178
051201060904
178
040400010503
178
041000030703
178
051402010502
178
051402020102
178
051201020301
178
051201040603
178
051201060104
178
050800030204
178
041000040408
178
051202040302
178
051201100108
178
051201080202
178
071200011102
178
051401040601
178
051202011602
178
051202030504
178
051201011406
178
040500010806
178
050800011001
178
051202081202
178
051202080102
178
051201110602
178
051201050203
178
040500012203
178
071200010209
178
051401010703
178
051201110102
178
051202020604
178
051202030511
178
051202011110
178
051202010402
178
051201061309
1

040500011705
178
050800030715
178
051201030202
178
051201010102
178
051401040705
178
051202020903
178
050902030602
178
050800030404
178
051201040201
178
051402020304
178
051201110702
178
051201070307
178
051201070310
178
051201061201
178
040500011506
178
040400010601
178
051202070706
178
051202080501
178
051202040902
178
051202030704
178
071200010103
178
040500011004
178
040500011006
178
051202040801
178
051201080502
178
051201010902
178
071200021306
178
051201060608
178
050800030202
178
051202010102
178
051401041405
178
051202020706
178
051202020603
178
051202020104
178
051202010805
178
051201060405
178
051201060304
178
051202090201
178
051202090402
178
051201080604
178
071200011005
178
071200011304
178
051401010905
178
050800030705
178
051202080303
178
051202070903
178
051202040105
178
071200010807
178
040400010507
178
051201020103
178
051202090803
178
051202081206
178
051202080806
178
051202011405
178
051201030511
178
051401040605
178
051202050202
178
050800030503
178
050800030401
1

051201080304
178
051202090605
178
051202070904
178
051202050605
178
071200011313
178
051402010901
178
051202090102
178
051201070302
178
071200020405
178
040500011203
178
051402011005
178
051201110903
178
051202020506
178
051202030811
178
051201100604
178
071200020203
178
041000030505
178
050800030601
178
051202050303
178
051202050501
178
051202011004
178
051202011104
178
050902030301
178
051201010601
178
051202030405
178
051201070409
178
051201060204
178
071200030303
178
051402020602
178
050902031004
178
051402010907
178
051202081505
178
051202081004
178
051201010801
178
071200020304
178
051201011102
178
051201111603
178
051202030805
178
051201111502
178
051201110405
178
041000040401
178
